# Chapter11. 심층 신경망 훈련

## 2. 사전훈련된 층 재사용하기(전이학습(transfer learning))

- **전이 학습(transfer learning)** : 일반적으로 아주 큰 규모의 DNN을 처음부터 새로 훈련하려고 하는 것보다, 해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아본 다음, 그 신경망의 하위층을 재사용하는 것이 좋음
    - 훈련 속도를 크게 높이고, 필요한 훈련 데이터도 크게 줄여줌
        - 만약 깊은 학습 모델에 적은 훈련 데이터를 사용하면 overfitting의 우려
    - 원래 문제에서 사용한 것과 크기가 다른 이미지를 입력으로 사용한다면 원본 모델에 맞는 크기로 변경하는 전처리 단계를 추가해야 함
    - 저수준 특성이 비슷한 입력에서 잘 작동
    - 상위층은 원본 작업에서 유용했던 특성과는 상당히 다를 수 있기 때문에 재사용할 층 개수를 잘 선정하는 것이 필요.(작업이 비슷하면 더 많은 층을, 아주 비슷한 작업이라면 모든 은닉층을 유지, 출력층만 교체)

- 방법
    1. 재사용하는 층을 모두 동결(경사하강법으로 가중치가 바뀌지 않도록 훈련되지 않는 가중치로 만듦)
    2. 모델을 훈련
    3. 성능을 평가
    4. 맨 위에 있는 한 두개의 은닉층의 동결을 해제하고 역전파를 통해 가중치를 조정하여 성능이 향상되는지 확인
        - 재사용 하지 않은 초기에 랜덤하게 초기화된 layer들이 초기에 재사용된 가중치를 망칠 수 있기 때문에, 적어도 처음 몇번의 에포크 동안은 재사용된 층을 동결하고 새로운 층을 학습하는 것이 좋음
        - 훈련 데이터가 많을수록 많은 층의 동결을 해제할 수 있음
        - 재사용 층의 동결을 해제할 때에는 가중치를 세밀하게 튜닝하기 위해 학습률을 줄이는 것이 좋음
        - 훈련 데이터셋이 적거나 성능이 좋지 않다면 상위 은닉층(들)을 제거하고 남은 은닉층을 다시 동결.

#### 구현

In [2]:
import numpy as np
from tensorflow import keras

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_a.layers[:-1]) # 출력층을 제외한 모든 layer 사용
model_B_on_A.add(keras.layers.Dense(1, activation = "sigmoid")) # 출력층 추가

-> model B를 훈련할 때, model A도 영향을 받음. 따라서 이를 원하지 않는다면 model_A를 클론해야함

In [ ]:
# 모델의 구조를 복사
model_A_clone = keras.models.clone_model(model_A)
# 모델의 가중치를 복사
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
# 재사용된 층을 동결
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

# 동결 해제 / 동결 후 반드시 다시 compile 해야함(compile 시에 훈련된 가중치를 모으기 때문)
model_B_on_A.compile(loss="binary_crossentropy", optimizer = "sgd",
                    metrics = ["accuracy"])

In [ ]:
# fit
history = model_B_on_A.fit(X_train_B, y_train_B, epochs = 4,
                          validation_data = (X_valid_B, y_valid_B))

# 동결 해제
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

# 세밀한 가중치 조정을 위해 학습률을 낮춤
optimizer = keras.optimizers.SGD(lr = 1e-4)

# 다시 compile해서 weight을 모음(model을 새로 지정한 것이 아니기 때문에 기존에 학습된 weight들은 그대로 있음)
model_B_on_A.compiile(loss="binary_corssentropy", optimizer = optimizer,
                     metrics = ["accuracy"])

#### 전이학습의 활용
- 작은 네트워크에서는 잘 작동하지 않음
    - 패턴 수를 적게 학습하기 때문
- 완전 연결 네트워크에서는 잘 작동하지 않음
    - 특정 패턴만을 학습하기 때문
- 심층 합성곱 신경망(CNN)에서 잘 작동
    - 좀 더 일반적인 특성을 (특히 아래쪽 층에서) 감지하는 경향이 있기 때문

### 비지도 사전훈련(unsupervised pretraining)

- 레이블된 훈련 데이터가 많지 않고, 비슷한 작업에 대해 훈련된 모델을 찾을 수 없다면 **비지도 사전훈련**을 수행할 수 있음
- 방법
    1. 레이블되지 않은 훈련 데이터를 많이 모음
    2. 오토인코더/GAN과 같은 비지도 학습 모델을 훈련(레이블이 없는 데이터/전체데이터로)
    3. 오토인코더/GAN 판별자의 하위층을 재사용하고 그 위에 새로운 작업에 맞는 출력층을 추가
    4. 지도학습으로 최종 네트워크를 세밀하게 튜닝

### 보조 작업에서 사전훈련
- 레이블된 훈련 데이터가 많지 않다면, 레이블된 훈련 데이터를 쉽게 얻거나 생성할 수 있는 보조 작업에서 첫 번째 신경망을 훈련. 그리고 이 신경망의 하위층을 실제 작업을 위해 재사용.
- 예를 들어, 개인 얼굴 인식 시스템에서 개인별 이미가 얼마 없다면, 좋은 분류기를 훈련하기에 충분하지 않음. 하지만 각 사람의 사진을 수백 개씩 모으기는 어려움. -> 인터넷에서 무작위로 많은 인물의 이미지를 수집해서 두 개의 다른 이미지가 같은 사람의 것인지 감지하는 첫 번째 신경망 훈련. -> 이런 신경망의 하위층을 재사용해 적은 양의 훈련 데이터에서 얼굴을 잘 구분하는 분류기 훈련

#### 자기지도학습(self-supervised learning)
- 데이터에서 스스로 레이블을 생성하고 지도 학습 기법으로 레이블된 데이터셋에서 모델을 훈련. 사람이 레이블을 부여할 필요가 없기 때문에 비지도학습의 형태로 분류
- 예를 들어, 수백만 텍스트 문서로 이루어진 말뭉치(corpus)에서 일부 단어를 랜덤하게 지우고 누락된 단어를 예측하는 모델을 훈련. 실제 작업에 이 모델을 재사용하고 레이블된 데이터를 사용하여 세부 튜닝